<a href="https://colab.research.google.com/github/RioVossKernan/Learning-AI/blob/main/PokemonNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize


In [ ]:
# seed random numbers to make calculation
# deterministic (just a good practice)
np.random.seed(2)

In [ ]:
#import CSV data set
import io
from google.colab import files
uploaded = files.upload()

Saving Pokemon.csv to Pokemon.csv


In [ ]:
#turn CSV to dataframe
points = pd.DataFrame()
points = pd.read_csv(io.BytesIO(uploaded['Pokemon.csv']))

#fix data
points = points.drop(columns=['#','Name', 'Generation'])
points['Legendary'] = pd.factorize(points['Legendary'])[0]
points['Type 2'] = points['Type 2'].fillna('No Type')
points['Type 1'] = pd.factorize(points['Type 1'])[0]
points['Type 2'] = pd.factorize(points['Type 2'])[0]

#normalize
points /= points.max()

points

,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,0,0,318,45,49,49,65,65,45,0
1,0,0,405,60,62,63,80,80,60,0
2,0,0,525,80,82,83,100,100,80,0
3,0,0,625,80,100,123,122,120,80,0
4,1,1,309,39,52,43,60,50,65,0
...,...,...,...,...,...,...,...,...,...,...
795,11,5,600,50,100,150,100,150,50,1
796,11,5,700,50,160,110,160,110,110,1
797,10,16,600,80,110,60,150,130,70,1
798,10,12,680,80,160,60,170,130,80,1


In [ ]:
#Shrink Data Set
#not necessary
#this just removes points to make it easier to see if the algorithm is working
for ind, rows in points.iterrows():
  if (not rows['Legendary'] and np.random.randint(1,3) == 1):
    points = points.drop(index = ind)

In [ ]:
X = np.asarray(points.drop(columns = ['Legendary']))
y = np.asarray(points['Legendary'])

#Split the train/test data and sample randoms points
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3) #random state is like seed
y_train = np.reshape(y_train, (-1, 1))

print(y_train.shape)
print(X_train.shape)
print(y_train)
print(X_train)

In [ ]:
# sigmoid function
def nonlin(x,deriv):
    if(deriv==True):
        return x*(1-x)
    else:
      return 1/(1+np.exp(-x))

In [ ]:
# initialize weights randomly with mean 0
syn0 = 2*np.random.random((9,40)) - 1
syn1 = 2*np.random.random((40,30)) - 1
syn2 = 2*np.random.random((30,20)) - 1
syn3 = 2*np.random.random((20,1)) - 1


for iter in range(300000):
	# Feed forward through layers 0, 1, 2, and 3
    l0 = X_train
    l1 = nonlin(np.dot(l0,syn0),False)
    l2 = nonlin(np.dot(l1,syn1),False)
    l3 = nonlin(np.dot(l2,syn2),False)
    l4 = nonlin(np.dot(l3,syn3),False)


  # adjust l4 based on expected outputs
    l4_error = y_train - l4
    l4_delta = l4_error*nonlin(l4,True)

    #print progress error
    if (iter% 10000) == 0: #once every 10,000 loops
        print("Error:" + str(np.mean(np.abs(l4_error))))

  # adjust l3 based on feedback from l4
    l3_error = l4_delta.dot(syn3.T)
    l3_delta = l3_error * nonlin(l3,True)

  # adjust l2 based on feedback from l3
    l2_error = l3_delta.dot(syn2.T)
    l2_delta = l2_error * nonlin(l2,True)

  # adjust l1 based on feedback from l2
    l1_error = l2_delta.dot(syn1.T)
    l1_delta = l1_error * nonlin(l1,True)

  # adjust synapses
    syn3 += l3.T.dot(l4_delta)
    syn2 += l2.T.dot(l3_delta)
    syn1 += l1.T.dot(l2_delta)
    syn0 += l0.T.dot(l1_delta)

print("Done Training")

Error:0.3841243510201911
Error:0.5000002027981949
Error:0.5000005060532617
Error:0.5000002487483703
Error:0.5000000789393934
Error:0.5000001882793323
Error:0.5000001534305842
Error:0.5000001293351588
Error:0.5000001116645033
Error:0.5000000981403858
Error:0.5000000874483052
Error:0.5000000787761854
Error:0.5000000715950536
Error:0.5000000655454336
Error:0.5000000603744015
Error:0.5000000558987525
Error:0.5000000519824566
Error:0.500000048522321
Error:0.5000000454385735
Error:0.5000000426684972
Error:0.5000000401620207
Error:0.5000000378785934
Error:0.5000000357849234
Error:0.5000000338533104
Error:0.5000000320603907
Error:0.5000000303861726
Error:0.5000000288132808
Error:0.5000000273263431
Error:0.5000000259114794
Error:0.5000000245558511
Done Training


In [ ]:
l4 = l4_error.reshape(1,-1)
y = y.reshape(1,-1)

for i in range(len(l4[0])-1):
  if l4[0,i] <= .5:
    l4[0,i] = 0
  else:
    l4[0,i] = 1

correct_total = 0
correct_L = 0
correct_N = 0

for i in range(len(l4[0])-1):
  if l4[0,i] == y[0,i]:
    correct_total += 1
    if l4[0,i] == 1:
      correct_L += 1
    else:
      correct_N += 1

unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

print(np.round(l4))
print('Success Rate: ' , 100 * (correct_total/len(l4[0])) , '%')
print('Success Rate of Legends', 100* correct_L/dict(zip(unique, counts))[1],'%')

{0: 106, 1: 65}
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Success Rate:  63.23529411764706 %
Success Rate of Legends 0.0 %
